In [ ]:
# Flow_from_directory
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications.resnet import ResNet152
from tensorflow.keras.regularizers import L2
import pandas as pd


In [ ]:
IMAGE_SIZE = 224

# Test data
test_generator = ImageDataGenerator(rescale=1./255)
test = test_generator.flow_from_directory('data/cropped/mass/test/', target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=16, class_mode='binary')

In [ ]:
# load model
model = load_model('model.h5')

In [ ]:
# Create roc curve
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
import numpy as np

# Get the predictions from the model using the generator
predictions = model.predict(test)

# Get the true values from the generator
true_values = test.classes

# Get the predicted probabilities for each class
pred_probabilities = predictions

# Get the false positive rate and true positive rate
fpr, tpr, thresholds = roc_curve(true_values, pred_probabilities)

# Plot the ROC curve
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='ResNet152')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')

### Recall, percission and accuracy

In [ ]:
# helper funcitons
def get_recall_i(conf_mat, i):
    """
    Computes.
    """
    return conf_mat[i, i] / np.sum(conf_mat[:, i])


def get_all_recalls(conf_mat):
    """
    Computes All recalls
    """

    return [get_recall_i(conf_mat, i) for i in range(conf_mat.shape[0])]


def get_percision_i(conf_mat, i):
    """
    Computes.
    """
    return conf_mat[i, i] / np.sum(conf_mat[i, :])


def get_all_percisions(conf_mat):
    """
    Computes all percisions.
    """

    return [get_percision_i(conf_mat, i) for i in range(conf_mat.shape[0])]

def get_f1_scores(conf_mat):
    """
    Computes the f1 scores of the given confusion matrix.
    """
    recalls = get_all_recalls(conf_mat)
    percisions = get_all_percisions(conf_mat)
    f1_scores = []
    for i in range(len(recalls)):
        f1_scores.append(2 * (percisions[i] * recalls[i]) / (percisions[i] + recalls[i]))
    return f1_scores


def get_overall_accuracy(conf_mat):
    """
    Computes the overall accuracy of the given confusion matrix.
    """
    return np.trace(conf_mat) / np.sum(conf_mat)

def get_confusion_matrix(result):
    result["Rounded"] = result["Prediction"].round()
    
    # creates confusion matrix
    true_positive_fake = result[(result['Filename'].str.contains(
        'benign')) & (result['Rounded'] == 0)].count()[0]
    false_positive_fake = result[(result['Filename'].str.contains(
        'malignant')) & (result['Rounded'] == 0)].count()[0]

    true_positive_real = result[(result['Filename'].str.contains(
        'malignant')) & (result['Rounded'] == 1)].count()[0]
    false_positive_real = result[(result['Filename'].str.contains(
        'benign')) & (result['Rounded'] == 1)].count()[0]

    conf_matrix = np.matrix([
        [true_positive_fake, false_positive_fake],
        [false_positive_real, true_positive_real]
    ])
    
    return conf_matrix

In [ ]:
# predictions and evaluation
pred_res = model.predict(
            test,
            verbose=1)

In [ ]:
test_results = pd.DataFrame({
                "Filename": test.filenames,
                "Prediction": pred_res.flatten()
            })
test_results

In [ ]:
conf_matrix = get_confusion_matrix(test_results)
print("conf_matrix: \n", conf_matrix)
print("Recalls:", np.round(get_all_recalls(conf_matrix), 3))
print("Precisions:", np.round(get_all_percisions(conf_matrix), 3))
print("F1 Scores:", np.round(get_f1_scores(conf_matrix), 3))
print("Accurcy:", np.round(get_overall_accuracy(conf_matrix), 3))